In [1]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
columns = {0: 'text', 1: 'ner'}
corpus: Corpus = ColumnCorpus(data_folder = 'data_flair/',
                              column_format = columns,
                              encoding="ISO-8859-1",
                              train_file='data_prep_train.txt',
                              test_file='data_prep_test.txt',
                              dev_file='data_prep_val.txt')

2022-09-25 18:33:24,525 Reading data from data_flair
2022-09-25 18:33:24,525 Train: data_flair\data_prep_train.txt
2022-09-25 18:33:24,525 Dev: data_flair\data_prep_val.txt
2022-09-25 18:33:24,540 Test: data_flair\data_prep_test.txt


In [2]:
import pandas as pd
data = [[len(corpus.train), len(corpus.test), len(corpus.dev)]]
# Prints out the dataset sizes of train test and development in a table.
pd.DataFrame(data, columns=["Train", "Test", "Development"])

,Train,Test,Development
0,1850,159,634


In [4]:
import flair
from typing import List
from flair.trainers import ModelTrainer
from flair.models import SequenceTagger
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings

tag_type = 'ner'
tag_dictionary = corpus.make_label_dictionary('ner')

# For faster training and smaller models, we can comment out the flair embeddings.
# This will significantly affect the performance though.
embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train('/content/model/conllpp',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=50,
              embeddings_storage_mode='gpu')

2022-09-25 18:42:08,154 Computing label dictionary. Progress:


1850it [00:00, 39789.26it/s]

2022-09-25 18:42:08,213 Dictionary created for label 'ner' with 5 values: GeoMeth (seen 2471 times), GeoPetro (seen 1718 times), GeoLoc (seen 344 times), GeoTime (seen 114 times)


2022-09-25 18:42:12,525 SequenceTagger predicts: Dictionary with 17 tags: O, S-GeoMeth, B-GeoMeth, E-GeoMeth, I-GeoMeth, S-GeoPetro, B-GeoPetro, E-GeoPetro, I-GeoPetro, S-GeoLoc, B-GeoLoc, E-GeoLoc, I-GeoLoc, S-GeoTime, B-GeoTime, E-GeoTime, I-GeoTime
2022-09-25 18:42:12,650 ----------------------------------------------------------------------------------------------------
2022-09-25 18:42:12,650 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [02:14<00:00,  6.74s/it]

...

2022-09-25 19:34:56,747 Evaluating as a multi-label problem: False
2022-09-25 19:34:56,763 DEV : loss 0.02821298874914646 - f1-score (micro avg)  0.9676
2022-09-25 19:34:56,778 BAD EPOCHS (no improvement): 0


2022-09-25 19:34:57,731 ----------------------------------------------------------------------------------------------------
2022-09-25 19:34:57,731 loading file \content\model\conllpp\best-model.pt
2022-09-25 19:34:58,559 SequenceTagger predicts: Dictionary with 19 tags: O, S-GeoMeth, B-GeoMeth, E-GeoMeth, I-GeoMeth, S-GeoPetro, B-GeoPetro, E-GeoPetro, I-GeoPetro, S-GeoLoc, B-GeoLoc, E-GeoLoc, I-GeoLoc, S-GeoTime, B-GeoTime, E-GeoTime, I-GeoTime, <START>, <STOP>


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:26<00:00,  5.40s/it]

2022-09-25 19:35:25,865 Evaluating as a multi-label problem: False
2022-09-25 19:35:25,881 0.9717	0.9643	0.968	0.938
2022-09-25 19:35:25,881 
Results:
- F-score (micro) 0.968
- F-score (macro) 0.9347
- Accuracy 0.938

By class:
              precision    recall  f1-score   support

     GeoMeth     0.9907    0.9953    0.9930       215
    GeoPetro     0.9517    0.9583    0.9550       144
      GeoLoc     0.8947    0.7083    0.7907        24
     GeoTime     1.0000    1.0000    1.0000         9

   micro avg     0.9717    0.9643    0.9680       392
   macro avg     0.9593    0.9155    0.9347       392
weighted avg     0.9707    0.9643    0.9668       392

2022-09-25 19:35:25,881 ----------------------------------------------------------------------------------------------------


{'test_score': 0.9679897567221512,
 'dev_score_history': [0.6547870733917246,
  0.7717423567824281,
  0.7634004779788324,
  0.8334376956793988,
  0.8448823903369357,
  0.8789293495175848,
  0.898320610687023,
  0.9037267080745343,
  0.8898819023300351,
  0.9275618374558303,
  0.8933460681311683,
  0.9323995127892815,
  0.9348682217509845,
  0.9343110296364192,
  0.945273631840796,
  0.9513064133016627,
  0.9526907155529273,
  0.9535160905840286,
  0.9540846750149075,
  0.9554703627248599,
  0.9577632361689471,
  0.9506355305941472,
  0.9519230769230769,
  0.9542600896860987,
  0.9578134284016637,
  0.9620029455081002,
  0.9611078373600471,
  0.9601887348864642,
  0.9594074074074074,
  0.9630068067475586,
  0.9604754829123329,
  0.9600477042337507,
  0.963464938126105,
  0.9610085019055996,
  0.9578134284016637,
  0.9639401934916446,
  0.9632918886915334,
  0.963641738102276,
  0.9646017699115044,
  0.9648033126293996,
  0.9621145374449339,
  0.9644640234948606,
  0.9666568309235762,
  